## 1. Import Dependencies

In [1]:
# stuff from gym
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#helpers
import numpy as np
import random
import os

#stuff from stable baselines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

E:\anaconda3\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


## 2. Types of spaces

In [9]:
Discrete(3).sample()

1

In [12]:
Box(0,1, shape=(3,3)).sample()

array([[0.09067822, 0.23819503, 0.84387136],
       [0.4592315 , 0.20830125, 0.2523304 ],
       [0.93719894, 0.21703143, 0.25233933]], dtype=float32)

In [15]:
Tuple((Discrete(3),Box(0,1, shape=(3,3)))).sample()

(2,
 array([[0.3397607 , 0.03724467, 0.06978406],
        [0.43154442, 0.8433508 , 0.89047474],
        [0.93551767, 0.0895842 , 0.2892942 ]], dtype=float32))

In [17]:
Dict({'height':Discrete(2), 'speed':Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([71.91626], dtype=float32))])

In [19]:
MultiBinary(4).sample()

array([1, 1, 1, 1], dtype=int8)

In [23]:
MultiDiscrete([5,2,2]).sample()

array([4, 0, 1], dtype=int64)

## 3. Building an Environment
- Build an agent to give us the best shower possible
- Random temperature
- 37 to 39 degrees

In [52]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3) #tap up, tap down or tap unchanged
        #temp high and low
        self.observation_space = Box(low= np.array([0]), high=np.array([100])) 
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        #apply temp adj
        self.state += action-1
        
        #Decrease shower time
        self.shower_length -= 1
        
       #Calculate Reward
        if self.state >=37 and self.state <=39:
            reward = 1
        else:
            reward = -1
            
        if self.shower_length <=0:
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        #implement visualization
        pass
    def reset(self):
        self.state = np.array([38+ random.randint(-3,3)]).astype(float)
        self.shower_length= 60
        return self.state
        

In [53]:
env = ShowerEnv()

In [54]:
env.reset()

array([35.])

## 4. Test Environment

In [56]:
episodes = 5
for episode in range(1, episodes + 1):
  obs = env.reset()
  done = False
  score = 0

  while not done:
   # env.render()
    
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    score += reward
  print('Episode:{}, Score:{}'.format(episode, score))
env.close()

Episode:1, Score:-36
Episode:2, Score:-8
Episode:3, Score:2
Episode:4, Score:-36
Episode:5, Score:-26


## 5. Train Model

In [57]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log= log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [60]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -12.8    |
| time/              |          |
|    fps             | 2041     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -8.76        |
| time/                   |              |
|    fps                  | 1313         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0035967003 |
|    clip_fraction        | 0.037        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.697       |
|    explained_variance   |

## 6. Savel Model

In [68]:
shower_path = os.path.join('Training','Saved Models', 'Shower_Model_PPO')

In [69]:
model.save(shower_path)

In [71]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(-24.0, 54.99090833947008)